In [ ]:
!pip install rcsbsearchapi biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.1/202.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rcsbsearchapi: filename=rcsbsearchapi-1.5.1-py2.py3-none-any.whl size=177323 sha256=3833025fa2c98a2f268fe41ee3fa067c00a13b02e380f09071645438e5613f52
  Stored in directory: /root/.cache/pip/wheels/44/ad/41/e8ae2bfcdd25a94bf4e51bbc8a953ba420d6a4d5977e1ceb4e
Successfully built rcsbsearchapi


In [ ]:
import rcsbsearchapi

In [ ]:
import requests

def fetch_fasta_sequence(pdb_id):
    # Define the URL for fetching sequence data
    url = f'https://www.rcsb.org/fasta/entry/{pdb_id}'

    try:
        # Make a GET request to fetch the sequence data
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        # Extract the FASTA sequence from the response content
        fasta_sequence = response.text

        return fasta_sequence
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None

def scrape_ids(polyResults):
  ids = [polyid.split("_")[0] for polyid in polyResults("polymer_entity")]
  return ids


### Build queries

Looking for MSSSP sequence in homo sapiens TRANSALDOLASE

In [ ]:
from pathlib import PurePosixPath
from rcsbsearchapi.search import SeqMotifQuery, TextQuery
#from rcsbsearchapi import rcsb_attributes as attrs

# Create terminals for each query (start w Homo sapiens)
q1 = TextQuery('"HOMO"')
print("Found ", q1.count())


# Use SeqMotifQuery class and add parameters
#results = SeqMotifQuery("C-x(2,4)-C-x(3)-[LIVMFYWC]-x(8)-H-x(3,5)-H.",
#                        pattern_type="prosite",
#                        sequence_type="protein")
# By default, the pattern_type argument is "simple" and the sequence_type argument is "protein".
q2 = SeqMotifQuery("MSSSP")  # X is used as a "variable residue" and can be any amino acid.
print("Found ", q2.count())


from rcsbsearchapi.search import TextQuery
#from rcsbsearchapi import rcsb_attributes as attrs
# Create terminals for each query
q3 = TextQuery('"TRANSALDOLASE"')
print("Found ", q3.count())

results = q1 & q2 & q3
# results("polymer_entity") produces an iterator of IDs with return type - polymer entities
results("polymer_entity")
print("Found ", results.count())
#for polyid in rresults("polymer_entity"):
#    print(polyid)


Found  137521
Found  32
Found  280
Found  1


In [ ]:
# process hits
ids = scrape_ids(results)


for id in ids:
  fasta_seq = fetch_fasta_sequence(id)

  if "Homo" in fasta_seq:
    print(fasta_seq)

>1F05_1|Chains A, B|TRANSALDOLASE|Homo sapiens (9606)
MSSSPVKRQRMESALDQLKQFTTVVADTGDFHAIDEYKPQDATTNPSLILAAAQMPAYQELVEEAIAYGRKLGGSQEDQIKNAIDKLFVLFGAEILKKIPGRVSTEVDARLSFDKDAMVARARRLIELYKEAGISKDRILIKLSSTWEGIQAGKELEEQHGIHCNMTLLFSFAQAVACAEAGVTLISPFVGRILDWHVANTDKKSYEPLEDPGVKSVTKIYNYYKKFSYKTIVMGASFRNTGEIKALAGCDFLTISPKLLGELLQDNAKLVPVLSAKAAQASDLEKIHLDEKSFRWLHNEDQMAVEKLSDGIRKFAADAVKLERMLTERMFNAENGK



### Applied to EF-hand
https://upload.wikimedia.org/wikipedia/commons/e/ef/EF-hand-prediction.jpg

[DNS]-x-[DNS]-{FLIVWY}-[DNESTG]-[DNQGHRK]-{GP}-[LIVMC]-[DENQSTAGC]-x(2)-[ED]

In [ ]:
from pathlib import PurePosixPath
from rcsbsearchapi.search import SeqMotifQuery, TextQuery
#from rcsbsearchapi import rcsb_attributes as attrs

# Create terminals for each query
q1 = TextQuery('"HOMO"')
print("Found ", q1.count(), " from hoomans")


# Use SeqMotifQuery class and add parameters
prosite_pattern=r"[DNS]-x-[DNS]-{FLIVWY}-[DNESTG]-[DNQGHRK]-{GP}-[LIVMC]-[DENQSTAGC]-x(2)-[ED]"
q2 = SeqMotifQuery(prosite_pattern,
                   pattern_type="prosite",
                   sequence_type="protein")
# By default, the pattern_type argument is "simple" and the sequence_type argument is "protein".
#q2 = SeqMotifQuery("MSSSP")  # X is used as a "variable residue" and can be any amino acid.
print("Found ", q2.count(), " w EF-hand")


# Create terminals for each query
from rcsbsearchapi.search import TextQuery
proteins = ["Myosin","Titin","Obscurin","Nebulin","Myomesin"]
#proteins = ["Titin","Obscurin","Nebulin","Myomesin"]
q3 = TextQuery(proteins[0])
#q3 = proteins[0]
for protein in proteins[1:]:
  q3 = q3 | TextQuery(protein)
  #q3 += " | "+protein

print("Found ", q3.count(), " that was among list of protein names")
#[print (x) for x in q3("polymer_entity")]

results = q1 & q2 & q3
# results("polymer_entity") produces an iterator of IDs with return type - polymer entities
print("Found ", results.count(), " that satisfy all conditions")
#for polyid in results("polymer_entity"):
#    print(polyid)


Found  137521  from hoomans
Found  4678  w EF-hand
Found  3649  that was among list of protein names
Found  434  that satisfy all conditions


In [ ]:
len(ids)

434

Get unique IDs

In [ ]:
 # process hits
ids = scrape_ids(results)
#print(len(ids))
# insert the list to the set
list_set = set(ids)
# convert the set to the list
ids = (list(list_set))
#print(len(ids))
print(ids)
len(ids)

['8SIK', '4UMO', '8AHS', '2LQC', '1CTR', '1XFY', '1PK0', '5J03', '6X35', '8J05', '2JZI', '7U9T', '1XFW', '7XNI', '1ZUZ', '2LL7', '1O1B', '8ACT', '6C1G', '4BYF', '8FMM', '8GLV', '6DMW', '8GM5', '6NUF', '8FMP', '1YRU', '7KO7', '7ZRQ', '6X33', '8I7O', '2MLF', '5GGM', '2F3Z', '1O1E', '6NUC', '6YNU', '7B1G', '5DOW', '7KOR', '6JI8', '7XN5', '1K90', '7CR7', '6JRS', '8EP0', '7BF1', '4CFR', '3HR4', '3GP2', '6GDK', '8FMR', '2PQ3', '5DSU', '2KFX', '7BYN', '1IH0', '8GM4', '7SX4', '8J04', '8BD2', '8FMT', '6MUD', '4L79', '5OEO', '1O1D', '7T2Q', '7VUU', '4GOW', '7WJI', '1LVC', '5WBX', '6C1D', '8SIN', '6Y94', '5DBR', '4GJF', '6EEB', '3BYA', '3RV5', '2KNE', '3ZWH', '2JT3', '6ZEG', '7SX3', '5JQA', '6K4K', '6AGI', '8TO0', '8FO6', '6Y4O', '6PAW', '5WC5', '7VVD', '7SVC', '6U3A', '7MIS', '6K7K', '7CR4', '1YRT', '1ZOT', '8OE0', '2VAY', '6GDL', '2W73', '2L7L', '3DVK', '6BUT', '7N6G', '6O5G', '6X32', '7VUT', '7XNL', '1LXF', '8DGH', '8OTZ', '6E2G', '1L7Z', '7SQC', '6HCS', '4BW8', '1WRL', '3LL8', '3SD6', '3EWT',

434

Grab the fasta sequences

In [ ]:
#ids = ids[0:3]
seqs=[]
for i,id in enumerate(ids):
  fasta_seq = fetch_fasta_sequence(id)
  seqs.append(fasta_seq)
  #print(len(seqs))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Check

In [ ]:
len(seqs)

434

In [ ]:
for seq in seqs:
  if "myomesin" in seq.lower():
    print(seq )

In [ ]:
import re
def words_contained(string, word_list,mode="all"):
    lc_word_list = [word.lower() for word in word_list]

    # Split the string into words
    words_in_string = re.split("\||\s", string)
    #words_in_string = string.split()
    lc_words_in_string = [word.lower() for word in words_in_string]

    #print(words_in_string)
    # Convert words_in_string to a set for faster lookup
    words_set = set(lc_words_in_string)

    # Check if all words in word_list are present in words_set
    if "all" in mode:
      for word in lc_word_list:
          if word not in words_set:
              #print("HERE")
              #print(word,words_set)
              return False
      return True
    elif "any" in mode:
      for word in lc_word_list:
          if word in words_set:
              #print("HERE")
              #print(word,words_set)
              return True
      return False
    else:
      raise RuntimeError("not understood")


#word_list = (["Homo","Troponin"])
all_word_list = (["Homo"])
#any_word_list = (["Parvalbumin"])
any_word_list = proteins

#ids = ids[0:3]
filteredSeqs=[]
for fasta_seq in seqs:
  if words_contained(fasta_seq, all_word_list,mode="all") is False:
    continue

  elif words_contained(fasta_seq, any_word_list,mode="any") is False:
    continue

  else:
    filteredSeqs.append(fasta_seq)

  print(fasta_seq)



>8ACT_1|Chains A, B|Myosin-7|Homo sapiens (9606)
DSEMAVFGAAAPYLRKSEKERLEAQTRPFDLKKDVFVPDDKQEFVKAKIVSREGGKVTAETEYGKTVTVKEDQVMQQNPPKFDKIEDMAMLTFLHEPAVLYNLKDRYGSWMIYTYSGLFCVTVNPYKWLPVYTPEVVAAYRGKKRSEAPPHIFSISDNAYQYMLTDRENQSILITGESGAGKTVNTKRVIQYFAVIAAIGDRSKKDQSPGKGTLEDQIIQANPALEAFGNAKTVRNDNSSRFGKFIRIHFGATGKLASADIETYLLEKSRVIFQLKAERDYHIFYQILSNKKPELLDMLLITNNPYDYAFISQGETTVASIDDAEELMATDNAFDVLGFTSEEKNSMYKLTGAIMHFGNMKFKLKQREEQAEPDGTEEADKSAYLMGLNSADLLKGLCHPRVKVGNEYVTKGQNVQQVIYATGALAKAVYERMFNWMVTRINATLETKQPRQYFIGVLDIAGFEIFDFNSFEQLCINFTNEKLQQFFNHHMFVLEQEEYKKEGIEWTFIDFGMDLQACIDLIEKPMGIMSILEEECMFPKATDMTFKAKLFDNHLGKSANFQKPRNIKGKPEAHFSLIHYAGIVDYNIIGWLQKNKDPLNETVVGLYQKSSLKLLSTLFANYAGADAPIEKGKGKAKKGSSFQTVSALHRENLNKLMTNLRSTHPHFVRCIIPNETKSPGVMDNPLVMHQLRCNGVLEGIRICRKGFPNRILYGDFRQRYRILNPAAIPEGQFIDSRKGAEKLLSSLDIDHNQYKFGHTKVFFKAGLLGLLEEMRDERLSRIITRIQAQSRGVLARMEYKKLLERRDSLLVIQWNIRAFMGVKNWPWMKLYFKIKPLLKSAEREKEMASMKEEFTRLKEALEKSEARRKELEEKMVSLLQEKNDLQLQVQAEQDNLADAEERCD
>8ACT_2|Chains C, D|Myosin light chain 3|Homo 

In [ ]:
 len( filteredSeqs )

8

In [ ]:
from Bio import PDB

def get_protein_name_from_pdb(pdb_id):
    # Create a PDBParser object
    parser = PDB.PDBParser(QUIET=True)

    # Fetch the structure from the PDB database
    # https://files.rcsb.org/download/8G4L.cif
    structure = parser.get_structure(pdb_id, f'https://files.rcsb.org/download/8G4L.cif')#'https://files.rcsb.org/download/{pdb_id}.cif')

    # Get the first model (assuming there's only one model)
    model = structure[0]

    # Get the first chain (assuming there's only one chain)
    chain = model[0]

    # Get the protein name from the first residue in the chain
    protein_name = chain.get_residues().__next__().get_full_id()[3][1]

    return protein_name

# Example PDB ID
pdb_id = "8G4L"

# Get the protein name from the PDB ID
protein_name = get_protein_name_from_pdb(pdb_id)
print(f"Protein Name for PDB ID {pdb_id}: {protein_name}")

FileNotFoundError: [Errno 2] No such file or directory: 'https://files.rcsb.org/download/8G4L.cif'

In [ ]:
https://files.rcsb.org/download/8G4L.cif

In [ ]:
from Bio import SeqIO
import re


def PStoRE(PrositePattern):
    rePattern = PrositePattern
    rePattern = rePattern.replace(' - ','')
    rePattern = rePattern.replace('-','')
    rePattern = rePattern.replace('x','.')
    rePattern = rePattern.replace('{','[^')
    rePattern = rePattern.replace('}',']')
    rePattern = rePattern.replace('(','{')
    rePattern = rePattern.replace(')','}')
    return rePattern



def search_prosite(pattern, sequence):
    #sequence = "DODRDGRLDGGE"
    g= PStoRE(pattern)
    PS00198compiled = re.compile(g)

    positions = PS00198compiled.findall(sequence)
    #matches = re.finditer(pattern, sequence)
    #positions = [match.start() for match in matches]
    return positions


def find_seq(pattern,fasta_string,verbose=True):
  # Parse the FASTA string
  #print(pattern)
  from io import StringIO
  records = SeqIO.parse(StringIO(fasta_string), "fasta")

  # Search for the PROSITE pattern in each sequence
  hits=0
  for record in records:
      sequence = str(record.seq)
      #print(sequence)
      # hack w known EF hand
      #sequence = "DODRDGRLDGGE"
      positions = search_prosite(pattern, sequence)
      if positions:
          hits+=1
          #help(record)
          print(record.description)
          print(f"Pattern found in sequence {record.id} at positions: {positions}")
      else:
          if verbose:
            print(f"Pattern not found in sequence {record.id}")
  return hits


def Debug():
  """
  Test of sequence parser
  """
  daSeq="""
>5JTH_1|Chain A|Calmodulin|Homo sapiens (9606)
MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPAFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK
  """
  efPattern=r"[DNS]-x-[DNS]-{FLIVWY}-[DNESTG]-[DNQGHRK]-{GP}-[LIVMC]-[DENQSTAGC]-x(2)-[ED]"
  #pattern=r"casdsdA"
  out = find_seq(efPattern,daSeq)
  print(out)

if 1:
  Debug()


5JTH_1|Chain A|Calmodulin|Homo sapiens (9606)
Pattern found in sequence 5JTH_1|Chain at positions: ['DKDGDGTITTKE', 'DKDGNGYISAAE', 'DIDGDGQVNYEE']
1


In [ ]:
def GetStructName(pdbCode):
  data = requests.get(f"https://data.rcsb.org/rest/v1/core/entry/{pdbCode}")
  result = data.json()
  return result["struct"]

In [ ]:
for filteredSeq in filteredSeqs:
  name = filteredSeq[1:5]
  print(name, " ", GetStructName(name)['title'] )
  find_seq(prosite_pattern,filteredSeq,verbose=False)
  print("\n")

8ACT   structure of the human beta-cardiac myosin folded-back off state
8ACT_3|Chains E, F|Myosin regulatory light chain 2, ventricular/cardiac muscle isoform|Homo sapiens (9606)
Pattern found in sequence 8ACT_3|Chains at positions: ['DQNRDGFIDKND']


5JQA   CaM:RM20 complex
5JQA_1|Chain A|Calmodulin|Homo sapiens (9606)
Pattern found in sequence 5JQA_1|Chain at positions: ['DKDGDGTITTKE', 'DADGNGTIDFPE', 'DKDGNGYISAAE', 'DIDGDGQVNYEE']


2LV6   The complex between Ca-Calmodulin and skeletal muscle myosin light chain kinase from combination of NMR and aqueous and contrast-matched SAXS data
2LV6_1|Chain A|Calmodulin|Homo sapiens (9606)
Pattern found in sequence 2LV6_1|Chain at positions: ['DKDGDGTITTKE', 'DADGNGTIDFPE', 'DKDGNGFISAAE', 'DIDGDGQVNYEE']


5TBY   HUMAN BETA CARDIAC HEAVY MEROMYOSIN INTERACTING-HEADS MOTIF OBTAINED BY HOMOLOGY MODELING (USING SWISS-MODEL) OF HUMAN SEQUENCE FROM APHONOPELMA HOMOLOGY MODEL (PDB-3JBH), RIGIDLY FITTED TO HUMAN BETA-CARDIAC NEGATIVELY STAINED THI

'8ACT'